In [ ]:
# === 0) Setup ===
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, classification_report
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB, ComplementNB


from scipy import sparse

SEED = 1234
rng = np.random.default_rng(SEED)
np.random.seed(SEED)

# Utils
def top_terms_for_doc(row_vector, idx2word, k=15):
    v = row_vector.toarray().ravel()
    if v.size == 0:
        return []
    top = np.argsort(v)[::-1][:k]
    return [(idx2word[i], float(v[i])) for i in top if v[i] > 0]


## 1) Carga de datos (train / test)


In [18]:
# Descarga 20 Newsgroups (texto en inglés). Quitamos headers/footers/quotes para enfocarnos en el contenido.
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=SEED)
test  = fetch_20newsgroups(subset='test',  remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=SEED)

print(f"Train docs: {len(train.data)} | Test docs: {len(test.data)} | #Clases: {len(train.target_names)}")
pd.DataFrame({"id": range(len(train.target_names)), "label": train.target_names}).head()


Train docs: 11314 | Test docs: 7532 | #Clases: 20


,id,label
0,0,alt.atheism
1,1,comp.graphics
2,2,comp.os.ms-windows.misc
3,3,comp.sys.ibm.pc.hardware
4,4,comp.sys.mac.hardware


## 2) Vectorización base (TF–IDF, `ngram_range=(1,1)` obligatorio)


In [19]:
# Vectorizador base: mantenemos ngram_range por consigna.
vectorizer = TfidfVectorizer(
    ngram_range=(1,1),
    lowercase=True,
    strip_accents='unicode'
    # (No cambiamos ngram_range; otros hiperparámetros se tunearán más adelante en el punto 3)
)

X_train = vectorizer.fit_transform(train.data)
X_test  = vectorizer.transform(test.data)
y_train = train.target
y_test  = test.target

vocab = vectorizer.vocabulary_
idx2word = {v:k for k,v in vocab.items()}

print(type(X_train), X_train.shape, X_test.shape)


<class 'scipy.sparse._csr.csr_matrix'> (11314, 101630) (7532, 101630)


## 3) Experimento 1 — Similaridad entre documentos
**Tarea.** Tomar 5 documentos al azar del *train* y medir la similaridad coseno contra el resto. Para cada uno, inspeccionar los **5 más similares** (índices, etiquetas y pequeños extractos) y evaluar si la similaridad tiene sentido por tema/etiqueta.


In [20]:
# Elegimos 5 documentos al azar
n_anchor = 5
anchor_ids = rng.choice(X_train.shape[0], size=n_anchor, replace=False)
anchor_ids


array([8754, 4965, 7404, 1009, 4899])

In [21]:
# Para cada ancla calculamos similitudes y mostramos el Top-5 (excluyendo el propio documento)
topk = 5
results_docs = []

for a in anchor_ids:
    sims = cosine_similarity(X_train[a], X_train).ravel()
    order = np.argsort(sims)[::-1]
    top_idx = [i for i in order if i != a][:topk]
    pack = {
        "anchor_id": int(a),
        "anchor_label": train.target_names[y_train[a]],
        "anchor_excerpt": train.data[a][:400].replace("\n"," "),
        "anchor_top_terms": top_terms_for_doc(X_train[a], idx2word, k=12),
        "neighbors": [
            {
                "doc_id": int(i),
                "sim": float(sims[i]),
                "label": train.target_names[y_train[i]],
                "excerpt": train.data[i][:200].replace("\n"," ")
            } for i in top_idx
        ]
    }
    results_docs.append(pack)

# Mostramos en una tabla resumida
rows = []
for r in results_docs:
    for nb in r["neighbors"]:
        rows.append({
            "anchor_id": r["anchor_id"],
            "anchor_label": r["anchor_label"],
            "neighbor_id": nb["doc_id"],
            "neighbor_label": nb["label"],
            "cosine": nb["sim"]
        })
df_doc_sims = pd.DataFrame(rows)
df_doc_sims.head(15)


,anchor_id,anchor_label,neighbor_id,neighbor_label,cosine
0,8754,talk.religion.misc,6552,talk.religion.misc,0.490405
1,8754,talk.religion.misc,10613,talk.religion.misc,0.481184
2,8754,talk.religion.misc,3616,talk.religion.misc,0.465348
3,8754,talk.religion.misc,8726,talk.politics.mideast,0.459895
4,8754,talk.religion.misc,3902,talk.religion.misc,0.459074
5,4965,comp.sys.mac.hardware,5830,comp.sys.mac.hardware,0.365328
6,4965,comp.sys.mac.hardware,9736,comp.sys.mac.hardware,0.361090
7,4965,comp.sys.mac.hardware,1822,comp.sys.ibm.pc.hardware,0.355371
8,4965,comp.sys.mac.hardware,2327,comp.sys.ibm.pc.hardware,0.340710
9,4965,comp.sys.mac.hardware,3408,comp.graphics,0.340530


In [22]:
# Vista enriquecida de un ejemplo para análisis cualitativo
i = 0  # cambia a 0..4 para ver cada ancla
r = results_docs[i]
print("ANCHOR:", r["anchor_id"], "| label:", r["anchor_label"])
print("Top terms:", [w for w,_ in r["anchor_top_terms"]])
print("Excerpt:", r["anchor_excerpt"], "\n")

for j,nb in enumerate(r["neighbors"], 1):
    print(f"#{j} -> id={nb['doc_id']}  label={nb['label']}  cosine={nb['sim']:.3f}")
    print("Excerpt:", nb["excerpt"])
    print("-"*80)


ANCHOR: 8754 | label: talk.religion.misc
Top terms: ['you', 'hudson', 'to', 'your', 'that', 'people', 'the', 'other', 'set', 'standard', 'moral', 'it']
Excerpt:  /(hudson) /If someone inflicts pain on themselves, whether they enjoy it or not, they /are hurting themselves.  They may be permanently damaging their body.  That is true.  It is also none of your business.    Some people may also reason that by reading the bible and being a Xtian you are permanently damaging your brain.  By your logic, it would be OK for them to come into your home, take away yo 

#1 -> id=6552  label=talk.religion.misc  cosine=0.490
Excerpt:  If I have a habit that I really want to break, and I am willing to make whatever sacrifice I need to make to break it, then I do so. There have been bad habits of mine that I've decided to put forth 
--------------------------------------------------------------------------------
#2 -> id=10613  label=talk.religion.misc  cosine=0.481
Excerpt: /(hudson) /Yes you do.  Who

**Interpretación (completar):**

- ¿Las etiquetas de los vecinos coinciden con la etiqueta del documento ancla? ¿Cuántas veces?
- ¿Los *top terms* de cada ancla explican por qué aparecen esos vecinos?
- ¿Observaste casos de *falsos amigos* (alta similaridad por términos ambiguos)?
- ¿Qué pasa con documentos muy cortos o muy largos?


## 4) Experimento 2 — Clasificador por prototipos (*zero-shot* 1-NN)
**Idea.** Para cada documento de *test*, buscamos su **vecino más similar** en *train* por coseno y **asignamos la etiqueta** de ese vecino.


In [23]:
# Usamos NearestNeighbors con métrica coseno (brute force) para 1-NN
nn = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='brute')
nn.fit(X_train)

# Para cada doc de test, obtenemos su vecino más cercano
dist, idx = nn.kneighbors(X_test, return_distance=True)
# Nota: similitud coseno = 1 - distancia_coseno
y_pred_proto = y_train[idx.ravel()]

f1_proto = f1_score(y_test, y_pred_proto, average='macro')
print("F1-macro (prototipos 1-NN por coseno):", f1_proto)

print("\nReporte de clasificación (resumen):")
print(classification_report(y_test, y_pred_proto, target_names=test.target_names, digits=3))


F1-macro (prototipos 1-NN por coseno): 0.504078533146506

Reporte de clasificación (resumen):
                          precision    recall  f1-score   support

             alt.atheism      0.366     0.508     0.425       319
           comp.graphics      0.544     0.481     0.510       389
 comp.os.ms-windows.misc      0.506     0.457     0.480       394
comp.sys.ibm.pc.hardware      0.340     0.538     0.417       392
   comp.sys.mac.hardware      0.535     0.499     0.516       385
          comp.windows.x      0.701     0.592     0.642       395
            misc.forsale      0.629     0.462     0.533       390
               rec.autos      0.607     0.518     0.559       396
         rec.motorcycles      0.635     0.515     0.569       398
      rec.sport.baseball      0.645     0.537     0.586       397
        rec.sport.hockey      0.748     0.722     0.735       399
               sci.crypt      0.552     0.588     0.570       396
         sci.electronics      0.531     0.328  

**Interpretación (completar):**

- ¿Qué clases acierta mejor el 1-NN? ¿Cuáles confunde? ¿Por qué?
- ¿Qué impacto esperás si normalizás o filtrás más el vocabulario?
- Ventajas/desventajas del enfoque 1-NN vs. modelos paramétricos.


## 5) Experimento 3 — Naïve Bayes (Multinomial y Complement) con *tuning* para F1-macro en *test*
**Consigna.** Mantener `ngram_range=(1,1)` y explorar otros hiperparámetros del **vectorizador** (p. ej. `min_df`, `max_df`, `strip_accents`, `sublinear_tf`, `stop_words`) y de **NB** (`alpha`, `fit_prior`). Comparar **MultinomialNB** vs **ComplementNB**.


In [24]:
# Construimos dos pipelines y probamos grids compactos (manteniendo ngram_range=(1,1))
base_vect = TfidfVectorizer(ngram_range=(1,1))

pipe_mnb = Pipeline([('vect', base_vect), ('clf', MultinomialNB())])
pipe_cnb = Pipeline([('vect', base_vect), ('clf', ComplementNB())])

grid_vect = {
    'vect__stop_words': [None, 'english'],
    'vect__min_df': [1, 3],
    'vect__max_df': [1.0, 0.95],
    'vect__strip_accents': ['unicode'],
    'vect__sublinear_tf': [False, True]
}
grid_mnb = {**grid_vect, 'clf__alpha': [0.1, 0.5, 1.0, 2.0], 'clf__fit_prior': [True, False]}
grid_cnb = {**grid_vect, 'clf__alpha': [0.1, 0.5, 1.0, 2.0]}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

g_mnb = GridSearchCV(pipe_mnb, grid_mnb, scoring='f1_macro', cv=cv, n_jobs=-1, verbose=1)
g_cnb = GridSearchCV(pipe_cnb, grid_cnb, scoring='f1_macro', cv=cv, n_jobs=-1, verbose=1)

g_mnb.fit(train.data, y_train)
g_cnb.fit(train.data, y_train)

print("Best MNB:", g_mnb.best_params_, "CV f1_macro:", g_mnb.best_score_)
print("Best CNB:", g_cnb.best_params_, "CV f1_macro:", g_cnb.best_score_)

# Evaluación en test
best_mnb = g_mnb.best_estimator_
best_cnb = g_cnb.best_estimator_

pred_mnb = best_mnb.predict(test.data)
pred_cnb = best_cnb.predict(test.data)

f1_mnb = f1_score(y_test, pred_mnb, average='macro')
f1_cnb = f1_score(y_test, pred_cnb, average='macro')

print("\nTest F1-macro — MultinomialNB:", f1_mnb)
print("Test F1-macro — ComplementNB:", f1_cnb)


Fitting 3 folds for each of 128 candidates, totalling 384 fits
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best MNB: {'clf__alpha': 0.1, 'clf__fit_prior': False, 'vect__max_df': 1.0, 'vect__min_df': 3, 'vect__stop_words': 'english', 'vect__strip_accents': 'unicode', 'vect__sublinear_tf': False} CV f1_macro: 0.7399645806985599
Best CNB: {'clf__alpha': 0.1, 'vect__max_df': 1.0, 'vect__min_df': 1, 'vect__stop_words': None, 'vect__strip_accents': 'unicode', 'vect__sublinear_tf': False} CV f1_macro: 0.7531021358710462

Test F1-macro — MultinomialNB: 0.6884692149765899
Test F1-macro — ComplementNB: 0.6953652590540836


In [25]:
print("=== Reporte MNB ===")
print(classification_report(y_test, pred_mnb, target_names=test.target_names, digits=3))
print("\n=== Reporte CNB ===")
print(classification_report(y_test, pred_cnb, target_names=test.target_names, digits=3))


=== Reporte MNB ===
                          precision    recall  f1-score   support

             alt.atheism      0.306     0.483     0.374       319
           comp.graphics      0.625     0.712     0.666       389
 comp.os.ms-windows.misc      0.656     0.566     0.608       394
comp.sys.ibm.pc.hardware      0.629     0.696     0.661       392
   comp.sys.mac.hardware      0.732     0.701     0.716       385
          comp.windows.x      0.810     0.754     0.781       395
            misc.forsale      0.792     0.762     0.776       390
               rec.autos      0.759     0.732     0.746       396
         rec.motorcycles      0.809     0.736     0.771       398
      rec.sport.baseball      0.934     0.821     0.874       397
        rec.sport.hockey      0.914     0.907     0.911       399
               sci.crypt      0.765     0.750     0.758       396
         sci.electronics      0.718     0.583     0.643       393
                 sci.med      0.860     0.758     0.805

**Interpretación (completar):**

- Comparar MNB vs CNB: ¿cuál rinde mejor y por qué (p. ej., **CNB** suele ser más robusto con clases desbalanceadas)?
- ¿Qué combinación de hiperparámetros mejoró más el F1 *macro*? Justificar.
- Referenciar el **baseline** con TF–IDF + MNB sin tuning para cuantificar la mejora.


## 6) Experimento 4 — Similaridad entre **palabras** (matriz término–documento)
Transponemos la matriz documento–término para obtener vectores de palabras y medimos similaridad coseno. **Elegimos manualmente** 5 términos interpretables y reportamos sus 5 términos más similares.


In [26]:
# Transponemos TF–IDF de train
TD = X_train.T.tocsr()  # término x documento

def similar_terms(term, topk=5):
    term = term.lower()
    if term not in vocab:
        return []
    i = vocab[term]
    sims = cosine_similarity(TD[i], TD).ravel()
    order = np.argsort(sims)[::-1]
    # saltamos el propio término (posición 0)
    neigh = [(idx2word[j], float(sims[j])) for j in order[1:topk+1]]
    return neigh

manual_terms = ["space", "graphics", "hockey", "windows", "god"]  # puedes cambiarlos manualmente
pairs = {t: similar_terms(t, topk=5) for t in manual_terms}
pairs


{'space': [('nasa', 0.3304220949572683),
  ('seds', 0.29664333833100454),
  ('shuttle', 0.29284482622528324),
  ('enfant', 0.2802695315319064),
  ('seti', 0.24649360843507817)],
 'graphics': [('comp', 0.2575688868332871),
  ('grieggs', 0.202647023284772),
  ('3d', 0.1973205380967198),
  ('cfd', 0.1944820634941295),
  ('discused', 0.19442638638066093)],
 'hockey': [('ncaa', 0.2742813641779748),
  ('nhl', 0.2652556525793872),
  ('affiliates', 0.24796636642669795),
  ('xenophobes', 0.2425667272476028),
  ('sportschannel', 0.22281945297261863)],
 'windows': [('dos', 0.3037048271255976),
  ('ms', 0.23204717301054048),
  ('microsoft', 0.22193665896525663),
  ('nt', 0.2140152182013542),
  ('for', 0.192975824156645)],
 'god': [('jesus', 0.26878884770685035),
  ('bible', 0.2616153851996155),
  ('that', 0.2560319560385339),
  ('existence', 0.2547707517690119),
  ('christ', 0.2510527028533356)]}

**Interpretación (completar):**

- Explicar por qué los términos vecinos son cercanos semánticamente (comparten contexto de clase: *sci.space*, *comp.graphics*, *rec.sport.hockey*, *comp.windows.x*, *talk.religion.misc*, etc.).
- ¿Aparecen términos muy frecuentes poco informativos? ¿Cómo afecta `max_df`/`min_df` o stop-words?


## 7) (Opcional) Guardado de resultados a CSV


In [27]:
# Exportamos las similitudes documento-documento resumidas
df_doc_sims.to_csv("doc_sims_top5.csv", index=False)

# Export de términos similares
rows = []
for t, lst in pairs.items():
    for w, s in lst:
        rows.append({"term": t, "neighbor": w, "cosine": s})
pd.DataFrame(rows).to_csv("term_sims_top5.csv", index=False)

print("Archivos generados: doc_sims_top5.csv, term_sims_top5.csv")


Archivos generados: doc_sims_top5.csv, term_sims_top5.csv


---
**Checklist de Entrega**
1. Tabla con Top-5 vecinos por cada uno de los 5 documentos ancla + comentarios cualitativos.
2. F1-macro del clasificador por prototipos (1-NN) + breve análisis de confusiones.
3. Tabla con mejores hiperparámetros y F1-macro de **MNB** y **CNB** en *test* + interpretación.
4. Lista de 5 términos elegidos manualmente y sus Top-5 vecinos + explicación.
